In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle
import warnings
warnings.filterwarnings('ignore')

In [8]:
# Read data
pathdata = '/home/daniu/Documentos/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/Users/danielarisaro/Documents/fundar/indice-mercado-trabajo-ingresos/'
pathdata = '/home/daniufundar/Documents/Fundar/indice-mercado-trabajo-ingresos/'

df_people_2021 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_21.txt', delimiter=';', low_memory=False)
df_people_2022 = pd.read_csv(pathdata + 'data_input/personas_tot_urb_3T_22.txt', delimiter=';', low_memory=False)

df_houses_2021 = pd.read_csv(pathdata + 'data_input/hogar_tot_urb_3T_2021.txt', delimiter=';', low_memory=False)
df_houses_2022 = pd.read_csv(pathdata + 'data_input/hogar_tot_urb_3T_2022.txt', delimiter=';', low_memory=False)

In [9]:
merged_df = pd.merge(df_people_2021[['CODUSU', 'COMPONENTE']], df_people_2022[['CODUSU', 'COMPONENTE']], on=['CODUSU', 'COMPONENTE'], how='outer', indicator=True)

# Duplicate rows and display the left-only rows (2021) and right-only rows (2022)
duplicated_rows = merged_df['_merge'].value_counts()['both']
left_only_rows = merged_df['_merge'].value_counts()['left_only']
right_only_rows = merged_df['_merge'].value_counts()['right_only']

# Select the right-only and right observation from both rows in merged_df
right_only_rows = merged_df[merged_df['_merge'] == 'right_only']
left_only_rows = merged_df[merged_df['_merge'] == 'left_only']
both_rows = merged_df[merged_df['_merge'] == 'both']
right_both_rows = pd.concat([both_rows, right_only_rows])

df_people_2021['rel_PONDERA'] = df_people_2021['PONDERA']/df_people_2021['PONDERA'].sum()
df_people_2022['rel_PONDERA'] = df_people_2022['PONDERA']/df_people_2022['PONDERA'].sum()

df_people_2021['rel_PONDII'] = df_people_2021['PONDII']/df_people_2021['PONDII'].sum()
df_people_2022['rel_PONDII'] = df_people_2022['PONDII']/df_people_2022['PONDII'].sum()

df_people_2021['rel_PONDIIO'] = df_people_2021['PONDIIO']/df_people_2021['PONDIIO'].sum()
df_people_2022['rel_PONDIIO'] = df_people_2022['PONDIIO']/df_people_2022['PONDIIO'].sum()

pob_2021_pondera = df_people_2021['PONDERA'].sum()
pob_2022_pondera = df_people_2022['PONDERA'].sum()

pob_2021_pondii = df_people_2021['PONDII'].sum()
pob_2022_pondii = df_people_2022['PONDII'].sum()

pob_2021_pondiio = df_people_2021['PONDIIO'].sum()
pob_2022_pondiio = df_people_2022['PONDIIO'].sum()

df_merged_with_copies = pd.concat([df_people_2021, df_people_2022])

# Option 2: Merge two dataframes erasing the first copy (keeping right only and right both observations)
df_people_2021_no_dupl = pd.merge(left_only_rows, df_people_2021, on=["CODUSU", "COMPONENTE"], how='inner')
df_people_2022_no_dupl = pd.merge(right_both_rows, df_people_2022, on=["CODUSU", "COMPONENTE"], how='inner')

df_merged_without_copies = pd.concat([df_people_2021_no_dupl, df_people_2022_no_dupl])

df_people_2021['PONDERA_new'] = df_people_2021['rel_PONDERA']/2 * (pob_2021_pondera + pob_2022_pondera)/2
df_people_2022['PONDERA_new'] = df_people_2022['rel_PONDERA']/2 * (pob_2021_pondera + pob_2022_pondera)/2

df_people_2021['PONDII_new'] = df_people_2021['rel_PONDII']/2 * (pob_2021_pondii + pob_2022_pondii)/2
df_people_2022['PONDII_new'] = df_people_2022['rel_PONDII']/2 * (pob_2021_pondii + pob_2022_pondii)/2

df_people_2021['PONDIIO_new'] = df_people_2021['rel_PONDIIO']/2 * (pob_2021_pondiio + pob_2022_pondiio)/2
df_people_2022['PONDIIO_new'] = df_people_2022['rel_PONDIIO']/2 * (pob_2021_pondiio + pob_2022_pondiio)/2

In [10]:
# Create new columns 'P21_new' and 'P47_new' initialized with NaN
df_merged_without_copies['P21_new'] = pd.NaT
df_merged_without_copies['P47T_new'] = pd.NaT
df_merged_without_copies['TOT_P12_new'] = pd.NaT

mask = (df_merged_without_copies['ANO4'] == 2021) & (df_merged_without_copies['P21'] != -9)
df_merged_without_copies.loc[mask, 'P21_new'] = df_merged_without_copies.loc[mask, 'P21'] / 0.546358573

mask = (df_merged_without_copies['ANO4'] == 2021) & (df_merged_without_copies['P47T'] != -9)
df_merged_without_copies.loc[mask, 'P47T_new'] = df_merged_without_copies.loc[mask, 'P47T'] / 0.546358573

mask = (df_merged_without_copies['ANO4'] == 2021) & (df_merged_without_copies['TOT_P12'] != -9)
df_merged_without_copies.loc[mask, 'TOT_P12_new'] = df_merged_without_copies.loc[mask, 'TOT_P12'] / 0.546358573

mask = df_merged_without_copies['ANO4'] == 2022
df_merged_without_copies.loc[mask, 'P21_new'] = df_merged_without_copies.loc[mask, 'P21']

mask = df_merged_without_copies['ANO4'] == 2022
df_merged_without_copies.loc[mask, 'P47T_new'] = df_merged_without_copies.loc[mask, 'P47T']

mask = df_merged_without_copies['ANO4'] == 2022
df_merged_without_copies.loc[mask, 'TOT_P12_new'] = df_merged_without_copies.loc[mask, 'TOT_P12']

In [11]:
df_merged_without_copies.to_csv(pathdata + 'data_output/Base_pool_solo_con_replicas_actuales.csv')